In [1]:
import datetime
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
full_demand = pd.read_csv('./raw/household_power_consumption.txt', sep=';', low_memory=False)

In [4]:
full_demand

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0
...,...,...,...,...,...,...,...,...,...
2075254,26/11/2010,20:58:00,0.946,0.000,240.430,4.000,0.000,0.000,0.0
2075255,26/11/2010,20:59:00,0.944,0.000,240.000,4.000,0.000,0.000,0.0
2075256,26/11/2010,21:00:00,0.938,0.000,239.820,3.800,0.000,0.000,0.0
2075257,26/11/2010,21:01:00,0.934,0.000,239.700,3.800,0.000,0.000,0.0


In [5]:
demand = full_demand[['Date', 'Time', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']].copy()
demand.loc[:,'Datetime'] = pd.to_datetime(demand.loc[:, 'Date'] + ' ' + demand.loc[:,'Time'], dayfirst=True)
demand.set_index('Datetime', inplace=True)

demand.replace("?", np.nan, inplace=True)

# Convert the columns to numeric
demand = demand.apply(pd.to_numeric, errors='coerce')

demand = demand.loc[:, ['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']]

In [10]:
demand["normalization"] = demand["Global_active_power"] / (demand["Sub_metering_1"] + demand["Sub_metering_2"] + demand["Sub_metering_3"]+10e-12)

demand["flexible_demand_response"] = demand["Sub_metering_2"] * demand["normalization"]
demand["household_energy_demand"] = (demand["Sub_metering_1"] + demand["Sub_metering_3"]) * demand["normalization"]

In [11]:
household_energy_demand = demand.loc[:, ["household_energy_demand"]]
household_energy_demand.rename(columns={"household_energy_demand": "energy"}, inplace=True)

flexible_demand_response = demand.loc[:, ["flexible_demand_response"]]
flexible_demand_response.rename(columns={"flexible_demand_response": "energy"}, inplace=True)

In [13]:
print("Max flexible_demand_response: ", np.max(flexible_demand_response["energy"]))
print("Min flexible_demand_response: ", np.min(flexible_demand_response["energy"]))
print("Max household_energy_demand: ", np.max(household_energy_demand["energy"]))
print("Min household_energy_demand: ", np.min(household_energy_demand["energy"]))

Max flexible_demand_response:  8.583999999998824
Min flexible_demand_response:  0.0
Max household_energy_demand:  10.161999999998569
Min household_energy_demand:  0.0


In [14]:
flexible_demand_response_episodes = flexible_demand_response.groupby(pd.Grouper(freq='W'))
household_energy_demand_episodes = household_energy_demand.groupby(pd.Grouper(freq='W'))

In [15]:
flexible_demand_response_episodes.describe()

energy                                                       
              count      mean       std  min  25%  50%       75%       max
Datetime                                                                  
2006-12-17   1836.0  0.469295  1.019370  0.0  0.0  0.0  0.247471  7.064000
2006-12-24  10078.0  0.266169  0.731060  0.0  0.0  0.0  0.112450  6.824000
2006-12-31  10078.0  0.363128  0.926618  0.0  0.0  0.0  0.094079  6.892000
2007-01-07  10080.0  0.281474  0.759999  0.0  0.0  0.0  0.065647  5.773950
2007-01-14  10079.0  0.245486  0.696839  0.0  0.0  0.0  0.135200  7.075517
...             ...       ...       ...  ...  ...  ...       ...       ...
2010-10-31  10080.0  0.157092  0.391254  0.0  0.0  0.0  0.182024  4.080792
2010-11-07  10080.0  0.229016  0.565837  0.0  0.0  0.0  0.220000  5.032000
2010-11-14  10080.0  0.164271  0.442792  0.0  0.0  0.0  0.138098  4.609287
2010-11-21  10080.0  0.166476  0.470079  0.0  0.0  0.0  0.094028  5.794571
2010-11-28   7023.0  0.168507  0.443197  0.0  0.0  0.0  0.084865  4.004464

[207 rows x 8 columns]

In [16]:
flexible_demand_response_episodes = pd.concat([group.fillna(0) for name, group in flexible_demand_response_episodes if len(group.dropna()) >= 10000 and group.index[0] >= datetime(2007, 1, 1) and group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))
household_energy_demand_episodes = pd.concat([group.fillna(0) for name, group in household_energy_demand_episodes if len(group.dropna()) >= 10000 and group.index[0] >= datetime(2007, 1, 1) and group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))

In [17]:
flexible_demand_response_episodes.describe()

energy                                                       
              count      mean       std  min  25%  50%       75%       max
Datetime                                                                  
2007-01-07  10080.0  0.281474  0.759999  0.0  0.0  0.0  0.065647  5.773950
2007-01-14  10080.0  0.245462  0.696809  0.0  0.0  0.0  0.135200  7.075517
2007-01-21  10080.0  0.249171  0.727400  0.0  0.0  0.0  0.077871  6.494098
2007-01-28  10080.0  0.228327  0.694452  0.0  0.0  0.0  0.000000  6.264186
2007-02-04  10080.0  0.210732  0.654384  0.0  0.0  0.0  0.073480  5.327448
...             ...       ...       ...  ...  ...  ...       ...       ...
2008-11-30  10080.0  0.141673  0.487970  0.0  0.0  0.0  0.000000  6.519091
2008-12-07  10080.0  0.131650  0.455512  0.0  0.0  0.0  0.000000  5.967165
2008-12-14  10080.0  0.161103  0.599055  0.0  0.0  0.0  0.000000  7.136000
2008-12-21  10080.0  0.182620  0.649132  0.0  0.0  0.0  0.000000  5.879348
2008-12-28  10080.0  0.102428  0.367824  0.0  0.0  0.0  0.000000  4.428000

[104 rows x 8 columns]

In [18]:
with pd.HDFStore('./minutely/flexible_demand_response.h5') as store:
    # Save each DataFrame with a key
    i = 0
    for name, group in flexible_demand_response_episodes:
        if not group.empty:
            store[f'eps_{i}'] = group
            i += 1

with pd.HDFStore('./minutely/household_energy_demand.h5') as store:
    # Save each DataFrame with a key
    i = 0
    for name, group in household_energy_demand_episodes:
        if not group.empty:
            store[f'eps_{i}'] = group
            i += 1

In [ ]:
# sub_metering_1: 